In [106]:
import pandas as pd
import requests
import datetime
import json

from config import Cfg as cfg

In [4]:
races_pos = pd.read_csv('./../data/geo-data/races_lat_lon.csv')
races_pos.head()

,race,lat,lon
0,alagoas70.3,-9.661669,-35.701667
1,arizona,33.433518,-111.937962
2,Arizona70.3,33.432270,-111.932982
3,astana70.3,51.125358,71.453152
4,atlanticcity70.3,39.350119,-74.449387


In [113]:
races_history = {}
with open('./../data/races/races.jl', 'r') as f:
    for line in f.readlines():
        race_result = json.loads(line)
        if races_pos.race.str.contains(f"^{race_result['id']}$").sum():
            date_race = datetime.datetime.strptime(
                f"{race_result['month']} {race_result['day']} {race_result['year']}", '%B %d %Y'
            )
            to_save = {
                'id': race_result['id'],
                'date': date_race.strftime("%Y-%m-%d")
            }
                                    
            exist = races_history.get(race_result['id'], False)
            if exist:
                races_history[race_result['id']]['editions'].append(to_save)
            else:
                races_history[race_result['id']] = {'editions': [to_save]}
    

In [121]:
# query the Darksky API to get historical data

query = False

if query:
    for race in races_history:
        race_data = races_history[race]
        n_years = len(race_data['editions'])
        race_data['n_editions'] = n_years
        for i in range(n_years):
            date = race_data['editions'][i]['date']
            lat,lon = races_pos.loc[races_pos.race == race, ['lat', 'lon']].to_numpy()[0]
            requestString = f"https://api.darksky.net/forecast/{cfg.darksky_secret}/{lat},{lon},{date}T12:00:00?exclude=currently,flags"
            if race_data['editions'][i].get('weather', 0) == 0:
                r = requests.get(requestString)
                response = r.json()
                race_data['editions'][i]['weather'] = response
                done += 1
            else:
                done += 1
else:
    done = 969
    

In [126]:
len(race)
n = 0
for result in races_history.values():
    n += len(result['editions'])
    
n == done

True

In [123]:
# races_history

In [124]:
with open('./../data/geo-data/races_weather.json', 'w') as f:
    f.write(json.dumps(races_history))